In [1]:
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout
from tqdm import tqdm

/home/derek/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/derek/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# read in data
cardtext = [list(x) for x in list(np.load('data/card_texts.npy'))]
c2i = np.load('data/c2i.npy').item()
i2c = np.load('data/i2c.npy').item()

In [3]:
# test - randomize!
np.random.seed = 1337
indices = list(np.random.permutation(len(cardtext)))
cardtext = [cardtext[i] for i in indices]
# cardtext = cardtext

In [4]:
# set parameters

DROP_RATE = 0.25 # dropout
EMBEDDING_SIZE = 200 # embedding size
HIDDEN_SIZE = 200 # lstm feature vector
HIDDEN_LAYERS = 2 # number of layers
START_EPOCH = 0
VOCAB_SIZE = len(c2i.keys()) # number of characters

WINDOW_SIZE = 50 # context length
CARDS_PER_BATCH = 5
NUM_EPOCHS = 1000

OUT_INCREMENT = 100 # printout after n batches - and save

In [5]:
def cardGenerator(cardtext, windowsize, cards_per_batch, c2i=c2i, debug=False):
    
    i = 0
    indices = list(np.random.permutation(len(cardtext)))
    idx = indices[i]
    
    def nextcard(cardtext, idx, debug=debug):
        if debug:
            card_idx = cardtext[idx]
        else:
            card_idx = [c2i[c] for c in cardtext[idx]]
        
        return list(card_idx)
    
    # pregenerate warmup sequence
    if debug:
        sequence = list(cardtext[idx][-(windowsize):])
    else:
        sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
    i += 1
    idx = indices[i]
    for j in range(cards_per_batch):
        sequence += nextcard(cardtext, idx)
        i += 1
        idx = indices[i]

    # create matrix
    x = []
    y = []
    
    # main iterator
    while True:
        
        # generate batch (of cards_per_batch cards)
        while len(sequence) > windowsize:
            x.append(np.array(sequence[:windowsize]))
            y.append(sequence[windowsize])
            sequence.pop(0)
        
        # generate batch_size worth of window-shifted data
        # reshape for sparse_categorical_crossentropy
        sequence = []
        y = np.array(y)
        y = y[:, np.newaxis]
        # yield and reset
        yield(np.asarray(x), y)
        x, y = [], []
        
        # check for too long, reset
        if len(indices[i:]) < cards_per_batch+1:
            indices = np.random.permutation(len(cardtext))
            i = 0
            idx = indices[i]
        else:
            i += 1
            idx = indices[i]
            
        # pregenerate warmup sequence
        if debug:
            sequence = list(cardtext[idx][-(windowsize):])
        else:
            sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
        i += 1
        idx = indices[i]
        for j in range(cards_per_batch):
            sequence += nextcard(cardtext, idx)
            i += 1
            idx = indices[i]

In [6]:
getbatch = cardGenerator(cardtext, WINDOW_SIZE, CARDS_PER_BATCH)

In [7]:
# define model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, 
                    batch_input_shape=(1, WINDOW_SIZE, )))
model.add(Dropout(DROP_RATE))
for _ in range(HIDDEN_LAYERS-1):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True, stateful=True))
model.add(LSTM(HIDDEN_SIZE, stateful=True))
model.add(Dense(VOCAB_SIZE, activation='softmax'))

In [8]:
# compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [9]:
# predict 'Ⓔ'

def predict(startchars='none', temperature=1.0, maxlen=300):
    
    seq_out = []
    
    if temperature=='random':
        tmp = np.random.random()
    else:
        tmp = temperature
    
    # starting sequence
    if startchars=='none':
        seq_in = [c2i['Ⓔ'] for i in range(WINDOW_SIZE)]
    
    elif startchars=='random':
        seq_in = []
        alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                 'w', 'x', 'y', 'z']
        alpha = [a for a in alpha if a in c2i.keys()]
        while len(seq_in) < WINDOW_SIZE-1:
            rnd = np.random.randint(0, len(alpha))
            seq_in += [c2i[alpha[rnd]]]
        seq_in += [c2i['Ⓔ']]
    
    else:
        s = list(startchars)
        s = s[:WINDOW_SIZE]
        seq_out =  [c2i[c] for c in s]
        while len(s) < WINDOW_SIZE:
            s.insert(0, 'Ⓔ')
        seq_in = [c2i[c] for c in s]
        
    # softmax temperature
    # scaling factor of logits = logits/temperature
    # high temp = more confident = more diverse, more mistakes
    # low temp: more conservative
    # https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras/37254117#37254117
    def sample(a, temperature=tmp):
        a = np.array(a)**(1/temperature)
        p_sum = a.sum()
        sample_temp = a/p_sum 

        # stupid fix for > 1 error
        while sum(sample_temp) > 1:
            sample_temp[0] -= 0.0001

        return np.argmax(np.random.multinomial(1, sample_temp, 1))

    for i in range(maxlen):

        # predict next char
        pred_out = model.predict(np.array(seq_in).reshape((1, WINDOW_SIZE)))
        # get index of highest pred
        idx = sample(pred_out[0])
        # save index for decoding
        seq_out.append(idx)
        # add index to input sequence
        seq_in.append(int(idx))
        # remove earliest
        seq_in.pop(0)

    # decode final sequence
    card_char = ''.join([i2c[int(i)] for i in seq_out])
    card_char = card_char.replace('·', '|')
    card_char = card_char.replace('Ⓔ', 'Ⓔ\n|')
    card_text = card_char.split('|')

    for f in card_text:
        print(f)
        
    return card_text

In [10]:
# # load model
# model.load_weights('model/temp-modelweights-epoch26-batch957.h5')
# START_EPOCH = 25

In [11]:
# just train, fix epochs later

# save card predictions
predictions = []

for epoch_idx in range(START_EPOCH, NUM_EPOCHS):
    # epochs are "meaningless" here, roughly batch size = 1 card so...
    for batch in tqdm(range(int(len(cardtext)/CARDS_PER_BATCH))):
        
        # get batch
        x_batch, y_batch = next(getbatch)
        
        # fit to card batch
        r = model.fit(x_batch, y_batch, epochs=1, batch_size=1, shuffle=False, verbose=0)

        # reset state
        model.reset_states()
        
        # if batch % OUT_INCREMENT == 0 and batch > 0:

        if batch % OUT_INCREMENT == 0 and batch > 0:
            model.save_weights('model/v2B-modelweights-epoch{}-cards{}.h5'.format(epoch_idx+1, CARDS_PER_BATCH*(batch)))
            print("EPOCH:", epoch_idx+1, "card #:", batch, "of", len(cardtext), r)
            predict(startchars='random', temperature='random')
            predict(startchars='random', temperature='random')
            predict(startchars='random', temperature='random')
            predictions.append("EPOCH "+str(epoch_idx+1)+"cards:"+str((batch)*CARDS_PER_BATCH)+": "+str(predict(startchars='random', temperature='random')))
            print('\n\n')

  2%|▏         | 100/5869 [1:37:25<93:40:38, 58.46s/it]

EPOCH: 1 card #: 100 of 29347 <keras.callbacks.History object at 0x7f38d8693eb8>
ng of the battlefield of turn.Ⓔ

ceching a creature pay put the battlefield then creature the battlefield then creature the battlefield target and target creature library.Ⓔ

cuman serresting ond target land onto the battlefield your mana controller then creature that card and intile ond target creature 
shan
③Ⓖ
C
creature
sorcery
⌧
soura
enchanted creature that card onto the battlefield then creature that card onto the battlefield then creature that card onto the battlefield then creature that card onto the battlefield then creature that card onto the battlefield then creature that card onto the ba
then creature of the battlefield target and that card onto the battlefield then creature target creature then creature libring creature gain 1 life target land onto the battlefield then creature the battlefield then your until end of turn.Ⓔ

balkerre
③Ⓖ
R
sorcery
⌧
creature target and onto the battlef


  2%|▏         | 101/5869 [1:39:00<94:13:49, 58.81s/it]

wald opns 3 linagra creature of library that cards of target and life sacrifice tapped card onto your only and tappod, sacrifice a creature that card into gains an mise
flying walked the battlefield untile target creature target spell/damasa, of to then card onto the beginning of turn.Ⓔ

binus
①
C
sor





  2%|▏         | 115/5869 [1:53:08<94:20:51, 59.03s/it]

KeyboardInterrupt: 

In [ ]:
# # todo: just save to json one time
# model.save('model/100test_model.h5')
# print("saved model to disk\n")
# model.save_weights('model/100test_model_weights.h5')
# print("saved model weights to disk\n")

In [ ]:
# # load model
# model.load_weights('model/100-modelweights-epoch71-batch99.h5')
# START_EPOCH = 25

In [ ]:
predict(startchars='random', temperature='random')